In [14]:
import PyMieScatt as ps
from astropy.io import fits
from astropy import units as u
from astropy.modeling.models import BlackBody
from lmfit import Parameters, minimize
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [42]:
class IceCont():
    def __init__(self,datafile):
        self.read_data(datafile)
        self.fit_poly()
        
    def read_data(self,datafile):
        colnames = ["Wavelength (um)","Flux (Jy)","Statistical Error (Jy)"]
        df = pd.read_csv(datafile,sep='\s+',skiprows=19,names=colnames)
        self.wave = df["Wavelength (um)"]
        self.fd = df["Flux (Jy)"]
        self.er = df["Statistical Error (Jy)"]

    def fit_poly(self):
        ranges = [(2.85,3.4),(3.6,4.0),(4.1,4.2)]
        fit_params = Parameters()
        fit_params.add('c0',value=1.0)
        fit_params.add('c1',value=1.0)
        fit_params.add('c2',value=0.0)
        fit_params.add('c3',value=0.0)
        fit_params.add('nd',value=1e-3)
 

In [43]:
file = "ChaINa2_L_06-05-2002.ascii"
IceCont('ChaINa2_L_06-05-2002.ascii')


0       2.81430
1       2.81577
2       2.81725
3       2.81872
4       2.82020
         ...   
1019    4.23960
1020    4.24083
1021    4.24205
1022    4.24328
1023    4.24450
Name: Wavelength (um), Length: 1024, dtype: float64
0       0.000000
1      -0.093840
2       0.093229
3       0.124195
4       0.139689
          ...   
1019   -0.096066
1020    0.668309
1021    2.783880
1022    4.600950
1023    0.000000
Name: Flux (Jy), Length: 1024, dtype: float64


In [8]:
file = "ChaINa2_L_06-05-2002.ascii"
IceCont(file)

OSError: Header missing END card.

In [ ]:

    def fit_poly(self):
        ranges = [(2.85,3.4),(3.6,4.0),(4.1,4.2)]
        fit_params = Parameters()
        fit_params.add('c0',value=1.0)
        fit_params.add('c1',value=1.0)
        fit_params.add('c2',value=0.0)
        fit_params.add('c3',value=0.0)
        fit_params.add('nd',value=1e-3)
        
        gsubs = []
        for range in ranges:
            gsubs = gsubs + np.where((self.wave>range[0]) & (self.wave<range[1]))[0].tolist()
        
        #self.cext_int -= np.min(self.cext_int[gsubs])
        self.cext_subs = self.cext_int[gsubs]
        
        out = minimize(self.residual, fit_params, args=(self.wave[gsubs],), kws={'data':self.fd[gsubs]})
        fit = self.residual(out.params, self.wave)
        cont_params = out.params
        cont_params['nd'].value = 0.0
        cont = self.residual(cont_params, self.wave)
        plt.plot(self.wave,self.fd)
        plt.plot(self.wave,fit)
        plt.plot(self.wave,cont)
        
        plt.show()
        
    def residual(self,pars,x,data=None,cext=None):
        cont = pars['c0'] + pars['c1']*x + pars['c2']*x**2 + pars['c3']*x**3
        if data is None:
            model = cont*np.exp(-self.cext_int*pars['nd'])
            return model
        else:
            model = cont*np.exp(-self.cext_subs*pars['nd'])
        return model-data

    def fit_bb(self):
        ranges = [(2.85,3.2),(3.65,4.0)]
        fit_params = Parameters()
        fit_params.add('const',value=1e7)
        fit_params.add('const_star',value=1e7)        
        fit_params.add('temp',value=1000,min=100,max=4000)
        fit_params.add('nd',value=1e-3,max=3e-3,min=0)
        
        gsubs = []
        for range in ranges:
            gsubs = gsubs + np.where((self.wave>range[0]) & (self.wave<range[1]))[0].tolist()
        
        #self.cext_int -= np.min(self.cext_int[gsubs])
        self.cext_subs = self.cext_int[gsubs]
        
        out = minimize(self.residual_bb, fit_params, args=(self.wave[gsubs],), kws={'data':self.fd[gsubs]})
        fit = self.residual_bb(out.params, self.wave)
        cont_params = out.params
        cont_params['nd'].value = 0.0
        cont = self.residual_bb(cont_params, self.wave)
        plt.plot(self.wave,self.fd)
        plt.plot(self.wave,fit)
        plt.plot(self.wave,cont)
        
        plt.show()
        
    def residual_bb(self,pars,x,data=None,cext=None):
        bb = BlackBody(temperature=pars['temp']*u.K)
        cont = pars['const']*bb(x*u.micron).value
        bb = BlackBody(temperature=4000*u.K)
        star = pars['const_star']*bb(x*u.micron).value
        if data is None:
            model = (star+cont)*np.exp(-self.cext_int*pars['nd'])
            return model
        else:
            model = (star+cont)*np.exp(-self.cext_subs*pars['nd'])
        return model-data
        
                

In [3]:
datafile = '/L_Spectra/ChaINa2_L_06-05-2002.ascii'
opacfile = 'jw_ice/opacities/'

In [ ]:
    def residual_bb(self,pars,x,data=None,cext=None):
        
        # temperature of continuum... fit to minimize residual
        bb = BlackBody(temperature=pars['temp']*u.K)
        cont = pars['const']*bb(x*u.micron).value
        
        # assume temperature of star=4000 K
        bb = BlackBody(temperature=4000*u.K)
        star = pars['const_star']*bb(x*u.micron).value
        
        # F = Fo e^(-tau)
        if data is None:
            model = (star+cont)*np.exp(-self.cext_int*pars['nd'])
            return model
        else:
            model = (star+cont)*np.exp(-self.cext_subs*pars['nd'])
        return model-data